# Local Beam Sudoku

In [53]:
import random

# definindo os elementos
digits = '123456789'
rows = 'ABCDEFGHI'
cols = digits


def cross(A, B):
    "Funcao auxiliar para realizar o produto cruzado dos elementos de A e B"
    return [a + b for a in A for b in B]


# cada square é um quadrado da matriz 9x9 do sudoku.
squares = cross(rows, cols)

# cada square tem/pertence 9 units(linha, coluna, grid/. Unitlist é a lista de todas as colunas, linhas e grids
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')])

# dicionario para retornar as units de um determinado square
units = dict((s, [u for u in unitlist if s in u]) for s in squares)

# peers de um square S sao todos os squares das 3 unidades relacionadas a S exceto o próprio S
peers = dict((s, set(sum(units[s], [])) - set([s])) for s in squares)


def grid_hash(grid):
    """Converte a grid em um dicionario de {square: valor} com '0' or '.' para os vazios."""
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))


def possible_squares(grid_dict):
    """Retorna uma lista embaralhada de squares livres"""
    possible = [k for k, v in grid_dict.items() if v in '0.']
    random.shuffle(possible)
    return possible

def valid_digits(key,grid_dict):
    """Retorna uma lista embaralhada de possiveis digitos"""
    valid = '123456789'
    for p in peers[key]:
        if grid_dict[p] not in '0.':
            valid = valid.replace(grid_dict[p],'')
    valid = list(valid)
    random.shuffle(valid)
    return ''.join(valid)

def generate_child(grid_dict):
    child = grid_dict.copy()
    pos = possible_squares(grid_dict)[0]
    value = valid_digits(pos,grid_dict)[0]
    #TODO: ver se é necessario fazer while até encontrar posição/value 
    if(pos and value):
        child[pos] = value

In [55]:
initial = """
    8 3 . |1 . . |6 . 5
    . . . |. . . |. 8 . 
    . . . |7 . . |9 . . 
    ------+------+------
    . 5 . |. 1 7 |. . . 
    . . 3 |. . . |2 . . 
    . . . |3 4 . |. 1 . 
    ------+------+------
    . . 4 |. . 8 |. . . 
    . 9 . |. . . |. . . 
    3 . 2 |. . 6 |. 4 7 
    """

grid = grid_values(initial)
possible_squares(grid)

['G8',
 'E8',
 'F3',
 'B2',
 'C9',
 'A5',
 'D4',
 'E1',
 'C5',
 'E4',
 'I7',
 'G2',
 'B1',
 'G5',
 'F7',
 'H9',
 'B5',
 'A3',
 'D3',
 'C6',
 'A8',
 'H7',
 'G1',
 'B6',
 'H3',
 'H4',
 'C1',
 'H6',
 'E5',
 'C3',
 'I4',
 'I5',
 'D9',
 'E2',
 'I2',
 'B3',
 'H5',
 'F6',
 'B7',
 'A6',
 'G7',
 'G9',
 'E9',
 'F2',
 'F1',
 'H1',
 'C2',
 'D7',
 'E6',
 'D8',
 'G4',
 'F9',
 'B4',
 'D1',
 'C8',
 'B9',
 'H8']

In [51]:
test = {'to have': True, 'to get': False, 'having': False}
print(test)
items = test.items()
random.shuffle(items)
OrderedDict(items)
print(test)

{'to have': True, 'to get': False, 'having': False}


TypeError: 'dict_items' object does not support indexing